In [1]:
import os
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [2]:
masterdata = spark.read.parquet("cleaned_data/Masterdata.parquet")
masterdata.createOrReplaceTempView("masterdata")

inbound_95 = spark.read.parquet("cleaned_data/Inbound_95%.parquet")
inbound_95.createOrReplaceTempView("inbound_95")

inbound_avg = spark.read.parquet("cleaned_data/Inbound_Avg.parquet")
inbound_avg.createOrReplaceTempView("inbound_avg")

inventory = spark.read.parquet("cleaned_data/Inventory.parquet")
inventory.createOrReplaceTempView("inventory")

picking_95 = spark.read.parquet("cleaned_data/Picking_95%.parquet")
picking_95.createOrReplaceTempView("picking_95")

picking_avg = spark.read.parquet("cleaned_data/Picking_Avg.parquet")
picking_avg.createOrReplaceTempView("picking_avg")

In [5]:
inbound_95_extended = inbound_95.join(masterdata, on="SKU", how="left")
inbound_95_extended.createOrReplaceTempView("inbound_95_extended")

inbound_avg_extended = inbound_avg.join(masterdata, on="SKU", how="left")
inbound_avg_extended.createOrReplaceTempView("inbound_avg_extended")

inventory_extended = inventory.join(masterdata, on="SKU", how="left")
inventory_extended.createOrReplaceTempView("inventory_extended")

picking_95_extended = picking_95.join(masterdata, on="SKU", how="left")
picking_95_extended.createOrReplaceTempView("picking_95_extended")

picking_avg_extended = picking_avg.join(masterdata, on="SKU", how="left")
picking_avg_extended.createOrReplaceTempView("picking_avg_extended")

inbound_95_extended.orderBy("SKU").show(10)

+--------------+-------------+-----------------+-------------------+-----------+-----------+----------+-----------+
|           SKU|Inbound_order|Quantity_Received|               Date|Weight [kg]|Length [cm]|Width [cm]|Height [cm]|
+--------------+-------------+-----------------+-------------------+-----------+-----------+----------+-----------+
|20001979173659|  23910068512|              1.0|2023-06-26 00:00:00|       0.17|      36.94|     25.74|       1.34|
|20008677611256|  23079354272|            168.0|2023-06-29 00:00:00|       0.06|      18.56|      15.1|        2.5|
|20008752513345|  23356244411|            300.0|2023-06-30 00:00:00|       0.34|      34.72|     23.33|       5.15|
|20010383425435|  23046229012|             25.0|2023-06-30 00:00:00|       0.21|      36.91|     25.82|       1.27|
|20018347188537|  23166053508|             10.0|2023-06-26 00:00:00|       0.84|      37.15|     22.59|      12.82|
|20020194938240|  23860509448|              2.0|2023-06-28 00:00:00|    

In [48]:
spark.sql("""
          SELECT COUNT(DISTINCT SKU) AS DISTINCT_SKUs FROM masterdata
          """).show()

+-------------+
|DISTINCT_SKUs|
+-------------+
|        86174|
+-------------+



In [53]:
#spark.sql("""SELECT * FROM masterdata ORDER BY SKU""").show(6)

In [52]:
spark.sql("""
    SELECT COUNT(*) AS DISTINCT_MODELS
    FROM(
        SELECT SUBSTRING(SKU, 1, 6) as sku_prefix,
            COUNT(DISTINCT SKU) as unique_skus
        FROM masterdata 
        GROUP BY SUBSTRING(SKU, 1, 6)
        ORDER BY sku_prefix
    )
""").show()

+---------------+
|DISTINCT_MODELS|
+---------------+
|          70060|
+---------------+

